In [1]:
import pandas as pd
import numpy as np
import polars as pl
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import gzip
import shutil
import pathlib
import os
import sqlalchemy
import sqlite3
import spacy
import re
import spacy_cleaner
from spacy_cleaner import processing, Cleaner
import re
import tqdm
from tqdm.notebook import tqdm, trange
import ipywidgets as widgets
from ipywidgets import IntProgress, HTML, VBox
from IPython.display import display
import time
import timeit
import sklearn

from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.nn.functional import softmax

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid_obj = SentimentIntensityAnalyzer()

%matplotlib inline
alt.data_transformers.disable_max_rows()


#spacy.prefer_gpu()
spacy.require_gpu(gpu_id=0)
nlp = spacy.load("en_core_web_sm")
#nlp = spacy.load("en_core_web_trf")

#nlp = spacy.load('/path/to/en_core_web_sm')

C:\Users\asl4a\AppData\Roaming\Python\Python310\site-packages\cupy\_environment.py:487: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda11x, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''


In [2]:
def print_files_in_directory(directory_path):
    with os.scandir(directory_path) as entries:
        for entry in entries:
            if entry.is_file():
                print(entry.name)

def get_sentiment_score(text):
    sentiment_scores = sid_obj.polarity_scores(text)
    return sentiment_scores

def clean_text(text):
    doc = nlp(text)

    cleaned_tokens = [token.lemma_.lower().strip() for token in doc if not token.is_punct and not token.is_space]
    cleaned_tokens = [token for token in cleaned_tokens if not nlp.vocab[token].is_stop]
    cleaned_text = ' '.join(cleaned_tokens)

    return cleaned_text

def remove_numbers(text):
    cleaner = Cleaner(
        nlp,
        processing.remove_number_token
    )
    return cleaner.clean(text)

def remove_numbers_regex(text):
    # Pattern to remove numbers from text data
    pattern = r"\d+"

    return re.sub(pattern,"",text)

In [3]:
conn = sqlite3.connect(r"C:/Users/asl4a/AirBnB_Data.db")
cursor = conn.cursor()

In [4]:
sql = """
SELECT reviews.id, reviews.temp_index, reviews.comments
FROM reviews"""

df_revs = pd.read_sql(sql,con=conn)

In [5]:
df_revs.head()

,id,temp_index,comments
0,1927,0,I can't say enough about how wonderful it was ...
1,3867,1,Michelle and Collier's home is wonderful! They...
2,4159,2,I spent one night at Michele's home and felt j...
3,5724,3,Michele and Collier are two of the loveliest p...
4,11891,4,We had the most lovely time staying with Miche...


In [6]:
df_revs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668045 entries, 0 to 9668044
Data columns (total 3 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   id          int64 
 1   temp_index  int64 
 2   comments    object
dtypes: int64(2), object(1)
memory usage: 221.3+ MB


In [7]:
sample_text = df_revs['comments'][:20]
sample_text

0     I can't say enough about how wonderful it was ...
1     Michelle and Collier's home is wonderful! They...
2     I spent one night at Michele's home and felt j...
3     Michele and Collier are two of the loveliest p...
4     We had the most lovely time staying with Miche...
5     My wife and I got back from a trip to Nashvill...
6     Michelle and Collier's home was safe and comfo...
7     We went from michelle and collier's home to th...
8     Staying with Michele and Collier made travelin...
9     No Words can really describe just how welcome ...
10    We stayed 3 nights at Michelle and Collier's b...
11    I stayed one night for a quick in and out trip...
12    My husband and I instantly felt like part of t...
13    Staying with Michele and Collier was the highl...
14    Staying with Michele and Collier was so wonder...
15    Coming back to Michele's house for the second ...
16    Staying with Collier and Michele was a true jo...
17    When you stay with Michele and Collier, ex

In [8]:
sample_text = remove_numbers(df_revs['comments'][:20])
sample_text

Cleaning Progress: 100%|██████████| 20/20 [00:01<00:00, 12.87it/s]


["I ca n't say enough about how wonderful it was to stay here . It was the highlight of our stay in Nashville ! Michele and her husband Collier felt like parents to my sister and I. They were so caring and helpful , giving us the best suggestions about places to eat and hang out in Nashville , and just being such warm , wonderful people that we had as much fun inside the house as we did going around Nashville ! Also , the house was absolutely beautiful , and our room had a giant window looking out onto their backyard , which leads directly into a gorgeous park that we walked through every day . We felt so relaxed and happy there , it felt like home . We did n't want to leave !",
 "Michelle and Collier 's home is wonderful ! They are both great people , Michele always nice and friendly on the phone and Collier was very sweet to tell us about the places to be in Nashville at night . With his advice my friend Raquel and I had a wonderful time in Nashville ! Everyone we met in their home w

In [11]:
# Use the function that utilizes regex instead of the function with Spacy unless you want to wait 3 hours for the code to finish running
#df_revs['cleaned_text'] = remove_numbers(df_revs['comments'])
df_revs['cleaned_text'] = df_revs['comments'].apply(remove_numbers_regex)

In [12]:
df_revs['comments'] = df_revs['comments'].astype(str)

In [13]:
len(df_revs)

9668045

In [14]:
#df_revs['cleaned_text'] = df_revs['cleaned_text'].apply(clean_text)
# This code has not yet run (as of 2/24/2024 14:46PM)
beg_slice = 8700000
end_slice = beg_slice + 1000000

total_iterations = len(df_revs['cleaned_text'][beg_slice:end_slice])
progress_bar = tqdm(total=total_iterations,desc='Processing text data...')

cleaned_text_list = []

for i in df_revs['cleaned_text'][beg_slice:end_slice]:
    cleaned_text_list.append(clean_text(i))
    progress_bar.update(1)

progress_bar.close()

Processing text data...:   0%|          | 0/3000000 [00:00<?, ?it/s]

In [15]:
cleaned_text_s = pd.Series(cleaned_text_list)
cleaned_text_s.to_csv(r"F:\Data Science\Datasets\Cleaned AirBnB Reviews\cleaned_reviews_16.csv")

In [21]:
#df_revs['cleaned_text'] = cleaned_text_list

100000